In [60]:
import pandas as pd
from pathlib import Path

In [61]:
# set project root and year
project_root = Path.cwd().parents[0]
year = 2024  # change as needed

# define input paths
raw_path = project_root / "data" / "raw" / str(year) / f"masc_{year}.xlsx"
summary_path = project_root / "data" / "reference" / "masc_summary.csv"
aci_path = project_root / "data" / "interim" / str(year) / f"aci_ag_summary_{year}.csv"
imputed_path = project_root / "data" / "interim" / str(year) / f"masc_imputed_{year}.csv"
crop_list_path = project_root / "data" / "reference" / "masc_crop_list.txt"
lut_path = project_root / "data" / "reference" / "crop_label_lut.csv"

# load data
masc_df = pd.read_excel(raw_path, engine="calamine")
summary_df = pd.read_csv(summary_path)
aci_df = pd.read_csv(aci_path)
imputed_df = pd.read_csv(imputed_path)
lut_df = pd.read_csv(lut_path)

# display basic info
print(f"Loaded {len(masc_df)} raw MASC rows for {year}")
print(f"Loaded {len(imputed_df)} imputed MASC rows for {year}")
print(f"Loaded {len(aci_df)} ACI summary rows for {year}")

Loaded 11389 raw MASC rows for 2024
Loaded 11388 imputed MASC rows for 2024
Loaded 31 ACI summary rows for 2024


In [62]:
lut_path = project_root / "data" / "reference" / "crop_label_lut.csv"

lut = pd.read_csv(lut_path)

print("Columns:", lut.columns.tolist())
print("Shape:", lut.shape)
lut.head(10)

Columns: ['Crop', 'Label']
Shape: (124, 2)


,Crop,Label
0,PINTO BEANS,Beans
1,BLACK BEANS,Beans
2,WHITE PEA BEANS,Peas
3,KIDNEY BEANS,Beans
4,OTH DRY EDIBLE BEANS,Beans
5,SMALL RED BEANS,Beans
6,ADZUKI BEANS,Beans
7,CRANBERRY BEANS,Beans
8,FABABEANS,Fababeans
9,FIELD PEAS,Peas


In [63]:
pd.options.display.float_format = '{:,.3f}'.format

In [64]:
imputed_df.head()

,year,rm,crop,variety,farms,acres,yield_per_acre,yield,imputed
0,2024,HARRISON PARK,ALFALFA/GRASS MIX.,NO VAR (VARIETY TYPE UNKNOWN),9,949.000,707.152,"671,087.248",0
1,2024,GLENBORO-SOUTH CYPRESS,PROC POTATOES-IRRIG,RUSSET BURBANK (GEMS),6,"2,971.000",19.311,"57,372.981",0
2,2024,NORFOLK-TREHERNE,PROC POTATOES-IRRIG,UMATILLA RUSSET,3,773.000,18.427,"14,244.071",0
3,2024,NORFOLK-TREHERNE,PROC POTATOES-IRRIG,RUSSET BURBANK (GEMS),3,696.000,18.169,"12,645.624",0
4,2024,MOUNTAIN,COARSE HAY,NO VAR (TYPE UNKNOWN),6,725.000,359.582,"260,696.950",0


In [65]:
aci_df.head()

,Label,pixel_count,hectares,acres
0,Canola/rapeseed,19277055,"1,734,934.950","4,287,117.626"
1,Spring wheat,18510889,"1,665,980.010","4,116,726.259"
2,Soybeans,7363359,"662,702.310","1,637,573.071"
3,Pasture/forages,4767195,"429,047.550","1,060,199.585"
4,Corn,3566267,"320,964.030","793,119.391"


In [66]:
# load
with open(crop_list_path) as f:
    masc_crops = [line.strip().upper() for line in f if line.strip()]

lut = pd.read_csv(lut_path)
lut["Crop"] = lut["Crop"].str.strip().str.upper()

# find unmapped crops
unmapped = sorted(set(masc_crops) - set(lut["Crop"]))

print(f"Unmapped crops: {len(unmapped)}")
for c in unmapped:
    print(c)

Unmapped crops: 0


In [67]:
imputed_df.columns.tolist()

['year',
 'rm',
 'crop',
 'variety',
 'farms',
 'acres',
 'yield_per_acre',
 'yield',
 'imputed']

In [68]:
# map MASC crops to ACI labels and compare totals

lut = pd.read_csv(lut_path)

lut["Crop"] = lut["Crop"].str.strip().str.upper()
lut["Label"] = lut["Label"].str.strip().str.lower()
imputed_df["crop"] = imputed_df["crop"].str.strip().str.upper()
aci_df["Label"] = aci_df["Label"].str.strip().str.lower()

# map masc crops to aci labels
masc_mapped = imputed_df.merge(lut, left_on="crop", right_on="Crop", how="left")

# sum MASC by label
masc_sum = masc_mapped.groupby("Label", as_index=False)["acres"].sum()

# sum ACI by label
aci_sum = aci_df.groupby("Label", as_index=False)["acres"].sum()

# merge for comparison
compare = masc_sum.merge(aci_sum, on="Label", how="outer", suffixes=("_masc", "_aci")).fillna(0)

# calculate difference
compare["acres_diff"] = compare["acres_aci"] - compare["acres_masc"]

print(f"Comparison table for {year}: {len(compare)} labels")
compare.head(50)


Comparison table for 2024: 33 labels


,Label,acres_masc,acres_aci,acres_diff
0,barley,"282,381.288","302,859.522","20,478.233"
1,beans,"159,549.220","171,383.693","11,834.474"
2,berries,0.000,232.847,232.847
3,buckwheat,"4,227.212",122.984,"-4,104.228"
4,canaryseed,"2,019.446","1,055.931",-963.515
5,canola/rapeseed,"3,251,711.343","4,287,117.626","1,035,406.283"
6,corn,"606,253.791","793,119.391","186,865.600"
7,fababeans,"3,257.273",532.636,"-2,724.637"
8,fallow,0.000,"2,151.003","2,151.003"
9,flaxseed,"22,196.488","12,807.274","-9,389.214"


In [69]:
# assume lut_df already exists
dupes = lut_df[lut_df.duplicated("Crop", keep=False)]
if len(dupes):
    display(dupes)

merged = recombined_df.merge(lut_df, left_on="crop", right_on="Crop", how="left")

unmatched = merged[merged["Label"].isna()]["crop"].unique()
print(f"Unmatched crops: {len(unmatched)}")
print(sorted(unmatched))

masc_label_summary = (
    merged.groupby("Label", as_index=False)["acres"]
    .sum()
    .rename(columns={"acres": "acres_masc"})
)

compare = masc_label_summary.merge(aci_summary_df, on="Label", how="outer")
compare["acres_diff"] = compare["acres_masc"] - compare["acres_aci"]
compare = compare.fillna(0).sort_values("acres_diff", ascending=False)
compare.head(40)

NameError: name 'recombined_df' is not defined